In [4]:
pip install TA_Lib-0.4.19-cp37-cp37m-win_amd64.whl

Processing c:\users\cz\deeppsx\deeppsx\module2\ta_lib-0.4.19-cp37-cp37m-win_amd64.whl
TA-Lib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy
import talib
import pandas as pd

### Testing Talib

In [6]:
close = numpy.random.random(100) * 100
output = talib.SMA(close, timeperiod = 21)
output

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
       43.63796136, 43.46168127, 39.61374087, 40.15486884, 40.43373265,
       40.3713111 , 42.10439442, 40.26485844, 44.27101404, 41.45853801,
       39.83749071, 42.70493772, 44.90585254, 47.74043866, 45.977706  ,
       45.85446805, 44.5383044 , 47.84420375, 48.60037653, 47.99215958,
       48.22834651, 47.5870453 , 46.43905231, 49.80285528, 49.87972585,
       48.51268675, 49.08963918, 50.8454976 , 51.95011896, 51.14834379,
       52.38819578, 53.13963202, 50.94807008, 48.99263202, 45.47048103,
       46.64481216, 45.2610338 , 49.42791643, 50.68387858, 53.29406026,
       51.46922397, 51.45769314, 50.93463488, 51.28426479, 49.04225031,
       47.42313907, 46.30539688, 43.28267118, 40.66154899, 43.82

### Simple Moving Average

In [7]:
def SMA(df):
  close = df['Close']
  
  output = talib.SMA(close, timeperiod=21)
  df['21 SMA'] = output 

  output = talib.SMA(close, timeperiod=50)
  df['50 SMA'] = output

  output = talib.SMA(close, timeperiod=100)
  df['100 SMA'] = output  

  output = talib.SMA(close, timeperiod=200)
  df['200 SMA'] = output

  return df 

### RSI

In [8]:
def RSI(df):
  close = df['Close']
  real = talib.RSI(close, timeperiod=14)
  df['14 RSI'] = real 

  return df

### Bollinger Bands

In [9]:
def bollinger(df):
  close = df['Close']
  upperband, middleband, lowerband = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
  df['Upper BBand'] = upperband
  df['Middle BBand'] = middleband
  df['Lower BBand'] = lowerband

  return df

### MACD

In [10]:
def macd(df):
  close = df['Close']
  macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
  trend = []
  for i in range(len(macdhist)):
      if (i <= 32):
          trend.append('NaN')
      else:
          if (macdhist[i] > 0):
              trend.append('Bullish')
          else:
              trend.append('Bearish')

  df['MACD'] = macd        
  df['MACD Signal'] = macdsignal
  df['MACD Trend'] = trend

  return df

### 52 Week High

In [11]:
def high(df):
  close = df['Close']
  high = []
  for i in range(len(df['Close'])):
      if (i >=260):
          temp = df['Close'][i-260:i]
          high.append(max(temp))
      else:
          high.append('NaN')

  df['52-week high'] = high

  return df

### 52 Week Low

In [12]:
def low(df):
  close = df['Close']
  low = []
  for i in range(len(df['Close'])):
      if (i >=260):
          temp = df['Close'][i-260:i]
          low.append(min(temp))
      else:
          low.append('NaN')

  df['52-week low'] = low

  return df

### Average Volume

In [13]:
def volume(df):
  close = df['Close']
  avg = []
  for i in range(len(df['Volume'])):
      if (i >=21):
          temp = df['Volume'][i-21:i]
          avg.append(sum(temp)/len(temp))
      else:
          avg.append('NaN')

  df['21-avg volume'] = avg

  return df


### Resistance

In [14]:
def resistance(df):
  close = df['Close']
  high = df['High']
  low = df['Low']
  pivot = (high+close+low)/3
  R1 = (2*pivot) - low
  R2 = pivot + (high - low)
  R3 = high + 2*(pivot - low)
  df['First Resistance'] = R1
  df['Second Resistance'] = R2
  df['Third Resistance'] = R3

  return df


### Support

In [15]:
def support(df):
  close = df['Close']
  high = df['High']
  low = df['Low']
  pivot = (high+close+low)/3
  S1 = (2*pivot) - high
  S2 = pivot - (high - low)
  S3 = low - 2*(high - pivot)
  df['First Support'] = S1
  df['Second Support'] = S2
  df['Third Support'] = S3

  return df

### Adding all indicators in the dataframe

In [16]:
def add_indicators(df):
  df = SMA(df)
  df = RSI(df)
  df = bollinger(df)
  df = macd(df)
  df = high(df)
  df = low(df)
  df = volume(df)
  df = resistance(df)
  df = support(df)

  return df

### Running on KSE100

In [ ]:
KSE100 = pd.read_csv("Input/KSE100.csv")

for i in range(len(KSE100)):
    
    print(i) 
    name = KSE100['Ticker'][i]
    load_name = 'Input/' + name +'.csv'
    df = pd.read_csv(load_name)

    df = add_indicators(df)
    df.to_csv('Output/Indicators/' + name +'.csv') 

In [19]:
def check(df1, df2):
  for i in range(len(df1)):
    if df1['Date'][i] == df2['Date'][0]:
      return i

### Merging indicators and Module 1 input

In [20]:
def merge(df1, df2):

  index = check(df1,df2)

  close = df1['Close'][index:].values
  df2['Original Price'] = close

  SMA21 = df1['21 SMA'][index:].values
  SMA50 = df1['50 SMA'][index:].values
  SMA100 = df1['100 SMA'][index:].values
  SMA200 = df1['200 SMA'][index:].values
  RSI = df1['14 RSI'][index:].values
  vol = df1['21-avg volume'][index:].values
  upper = df1['Upper BBand'][index:].values
  middle = df1['Middle BBand'][index:].values
  lower = df1['Lower BBand'][index:].values
  high = df1['52-week high'][index:].values
  low = df1['52-week low'][index:].values
  macd = df1['MACD'][index:].values
  macdsignal = df1['MACD Signal'][index:].values
  trend = df1['MACD Trend'][index:].values

  df2['21 SMA'] = SMA21
  df2['50 SMA'] = SMA50
  df2['100 SMA'] = SMA100
  df2['200 SMA'] = SMA200
  df2['RSI'] = RSI
  df2['21-avg volume'] = vol
  df2['Upper BBand'] = upper
  df2['Middle BBand'] = middle
  df2['Lower BBand'] = lower
  df2['52-week high'] = high
  df2['52-week low'] = low
  df2['MACD'] = macd
  df2['MACD Signal'] = macdsignal
  df2['MACD Trend'] = trend

  return df2

### Final KSE100 files

In [ ]:
for i in range(len(KSE100)):
    
    print(i) 
    name = KSE100['Ticker'][i]
    load_name = 'Output/Indicators/' + name +'.csv'
    df1 = pd.read_csv(load_name)
    load_name = 'Input/' + name +'.csv'
    df2 = pd.read_csv(load_name)

    df3 = merge(df1, df2)
    df3.to_csv('Output/' + name +'.csv') 